In [125]:
import torch
import json
from tqdm import tqdm
from itertools import product as p

# Load fake activations

In [49]:
num_neurons_d = {} # {fname, int}
representations_d = {} # {fname, tensor}
f1, f2, f3 = "foo", "bar", "baz"
representation_files = [f1, f2, f3]

In [50]:
# initialize `num_neurons_d`, `representations_d` with fake data
n1, n2, n3 = 100, 80, 70
nword = 1000
t1 = torch.randn(nword, n1)
t2 = torch.randn(nword, n2)
t3 = torch.randn(nword, n3)
num_neurons_d[f1] = n1
num_neurons_d[f2] = n2
num_neurons_d[f3] = n3
representations_d[f1] = t1
representations_d[f2] = t2
representations_d[f3] = t3

### Set `means_d`, `stdevs_d` loop

In [19]:
means_d = {}
stdevs_d = {}

In [6]:
# arbitrary loop variable
network = f1

In [8]:
t = representations_d[network]
means = t.mean(0, keepdim=True)
means.shape

torch.Size([1, 100])

In [17]:
stdevs = (t - means).pow(2).mean(0, keepdim=True).pow(0.5)
stdevs.shape

torch.Size([1, 100])

In [20]:
means_d[network] = means
stdevs_d[network] = stdevs

### For network, other_network in loop

In [84]:
# Set `means_d`, `stdevs_d`
means_d = {}
stdevs_d = {}

for network in tqdm(representations_d, desc='mu, sigma'):
    t = representations_d[network]

    means_d[network] = t.mean(0, keepdim=True)
    stdevs_d[network] = (t - means_d[network]).pow(2).mean(0, keepdim=True).pow(0.5)

In [ ]:
correlations = {network: {} for network in representations_d}
num_words = list(representations_d.values())[0].size()[0] # TO DO: make more elegant

In [87]:
# arbitrary loop variable
network = f1
other_network = f2

In [88]:
# if network == other_network:
#     continue

In [89]:
# if other_network in correlations[network].keys(): # TO DO: optimize?
#     continue

In [90]:
t1 = representations_d[network] # "tensor"
t2 = representations_d[other_network] 

m1 = means_d[network] # "means"
m2 = means_d[other_network]

In [91]:
covariance = (torch.mm(t1.t(), t2) / num_words # E[ab]
              - torch.mm(m1.t(), m2)) # E[a]E[b]

In [92]:
s1 = stdevs_d[network] # "stdevs"
s2 = stdevs_d[other_network]

correlation = covariance / torch.mm(s1.t(), s2)

In [93]:
correlation = correlation.cpu().numpy()

correlations[network][other_network] = correlation
correlations[other_network][network] = correlation.T

In [94]:
correlations[network][other_network].shape

(100, 80)

In [108]:
# Test against numpy. Great. 
import numpy as np
a1 = t1.numpy()
a2 = t2.numpy()
np.corrcoef(a1, a2, rowvar=False)[:n1, -n2:] - correlations[network][other_network]

array([[-8.82273012e-09, -1.24303668e-09,  4.47088536e-09, ...,
         1.38618124e-08,  1.82512717e-09, -4.95861063e-09],
       [-1.40548824e-08, -4.86076822e-09,  1.41226910e-08, ...,
         2.09053897e-09,  2.99413491e-09, -1.26816585e-08],
       [-6.18208061e-09,  1.20600668e-08, -9.40264474e-09, ...,
         5.83043260e-10,  5.81605704e-09,  1.09122010e-08],
       ...,
       [ 1.25254017e-08,  4.01380670e-09,  6.42151650e-09, ...,
        -2.86913503e-09,  2.39079806e-08, -2.91457821e-09],
       [-1.70283885e-09, -3.63192508e-09, -2.91479070e-10, ...,
         1.04786853e-08, -6.40657440e-10,  5.07498338e-09],
       [-1.23479733e-08, -8.25447331e-09,  2.17075951e-09, ...,
         2.24927934e-08, -8.89444305e-09, -1.90452193e-08]])

### Setting `clusters`

In [129]:
# Set `means_d`, `stdevs_d`
means_d = {}
stdevs_d = {}

for network in tqdm(representations_d, desc='mu, sigma'):
    t = representations_d[network]

    means_d[network] = t.mean(0, keepdim=True)
    stdevs_d[network] = (t - means_d[network]).pow(2).mean(0, keepdim=True).pow(0.5)

# Set `correlations`
correlations = {network: {} for network in representations_d}
num_words = list(representations_d.values())[0].size()[0] # TO DO: make more elegant

for network, other_network in tqdm(p(representations_d,
                                     representations_d),
                                   desc='correlate',
                                   total=len(representations_d)**2):

    if network == other_network:
        continue

    if other_network in correlations[network].keys(): # TO DO: optimize?
        continue

    t1 = representations_d[network] # "tensor"
    t2 = representations_d[other_network] 
    m1 = means_d[network] # "means"
    m2 = means_d[other_network]
    s1 = stdevs_d[network] # "stdevs"
    s2 = stdevs_d[other_network]

    covariance = (torch.mm(t1.t(), t2) / num_words # E[ab]
                  - torch.mm(m1.t(), m2)) # E[a]E[b]
    correlation = covariance / torch.mm(s1.t(), s2)
    
    correlation = correlation.cpu().numpy()
    correlations[network][other_network] = correlation
    correlations[other_network][network] = correlation.T

correlate: 100%|█| 9/9 [00:00<00:00, 4087.13it/s]


In [131]:
clusters = {network: {} for network in representations_d}

In [132]:
# Loop variable
network = f1
neuron = 0

In [136]:
{other : max(range(num_neurons_d[other]),
             key = lambda i: abs(correlations[network][other][neuron][i])) 
     for other in correlations[network]}

{'bar': 74, 'baz': 22}

### Set `neuron_sort`

In [140]:
# Set `clusters`
clusters = {network: {} for network in representations_d}
for network in tqdm(representations_d, desc='clusters', total=len(representations_d)):
    for neuron in range(num_neurons_d[network]): 
        clusters[network][neuron] = {
            other : max(range(num_neurons_d[other]),
                         key = lambda i: abs(correlations[network][other][neuron][i])) 
             for other in correlations[network]
        }

clusters: 100%|█| 3/3 [00:00<00:00, 66.24it/s]


In [141]:
clusters

{'foo': {0: {'bar': 74, 'baz': 22},
  1: {'bar': 3, 'baz': 63},
  2: {'bar': 48, 'baz': 35},
  3: {'bar': 6, 'baz': 22},
  4: {'bar': 20, 'baz': 10},
  5: {'bar': 68, 'baz': 47},
  6: {'bar': 6, 'baz': 12},
  7: {'bar': 17, 'baz': 58},
  8: {'bar': 11, 'baz': 23},
  9: {'bar': 20, 'baz': 52},
  10: {'bar': 56, 'baz': 35},
  11: {'bar': 51, 'baz': 60},
  12: {'bar': 1, 'baz': 52},
  13: {'bar': 67, 'baz': 54},
  14: {'bar': 20, 'baz': 55},
  15: {'bar': 10, 'baz': 65},
  16: {'bar': 36, 'baz': 41},
  17: {'bar': 70, 'baz': 8},
  18: {'bar': 55, 'baz': 12},
  19: {'bar': 46, 'baz': 1},
  20: {'bar': 17, 'baz': 7},
  21: {'bar': 64, 'baz': 39},
  22: {'bar': 1, 'baz': 65},
  23: {'bar': 38, 'baz': 43},
  24: {'bar': 22, 'baz': 5},
  25: {'bar': 16, 'baz': 2},
  26: {'bar': 31, 'baz': 50},
  27: {'bar': 34, 'baz': 62},
  28: {'bar': 9, 'baz': 64},
  29: {'bar': 2, 'baz': 37},
  30: {'bar': 71, 'baz': 54},
  31: {'bar': 51, 'baz': 27},
  32: {'bar': 68, 'baz': 15},
  33: {'bar': 46, 'baz': 

In [149]:
neuron_sort = {}
network = f1
op = max

In [148]:
sorted(
    range(num_neurons_d[network]),
    key = lambda i : op(
        abs(correlations[network][other][i][clusters[network][i][other]])
        for other in clusters[network][i]),
    reverse=True
)

[18,
 24,
 97,
 12,
 48,
 31,
 0,
 3,
 34,
 60,
 56,
 43,
 87,
 52,
 64,
 57,
 58,
 13,
 66,
 4,
 49,
 35,
 53,
 2,
 6,
 70,
 86,
 5,
 80,
 37,
 62,
 68,
 50,
 77,
 22,
 42,
 85,
 7,
 76,
 40,
 27,
 9,
 69,
 10,
 74,
 11,
 55,
 67,
 59,
 36,
 88,
 38,
 41,
 47,
 19,
 61,
 1,
 99,
 94,
 46,
 44,
 29,
 73,
 25,
 45,
 26,
 96,
 33,
 28,
 16,
 63,
 82,
 54,
 20,
 30,
 98,
 71,
 8,
 84,
 32,
 78,
 51,
 95,
 81,
 75,
 23,
 83,
 89,
 90,
 93,
 15,
 17,
 92,
 39,
 72,
 79,
 21,
 65,
 91,
 14]

In [150]:
neuron_sort = {} # {network, sorted_list}
# Sort neurons by worst (or best) best correlation with another neuron
# in another network.
for network in tqdm(representations_d, desc='annotation'):
    neuron_sort[network] = sorted(
        range(num_neurons_d[network]),
        key = lambda i : op(
            abs(correlations[network][other][i][clusters[network][i][other]])
            for other in clusters[network][i]),
        reverse=True
    )

annotation: 100%|█| 3/3 [00:00<00:00, 1362.97it/s]


In [151]:
neuron_sort

{'foo': [18,
  24,
  97,
  12,
  48,
  31,
  0,
  3,
  34,
  60,
  56,
  43,
  87,
  52,
  64,
  57,
  58,
  13,
  66,
  4,
  49,
  35,
  53,
  2,
  6,
  70,
  86,
  5,
  80,
  37,
  62,
  68,
  50,
  77,
  22,
  42,
  85,
  7,
  76,
  40,
  27,
  9,
  69,
  10,
  74,
  11,
  55,
  67,
  59,
  36,
  88,
  38,
  41,
  47,
  19,
  61,
  1,
  99,
  94,
  46,
  44,
  29,
  73,
  25,
  45,
  26,
  96,
  33,
  28,
  16,
  63,
  82,
  54,
  20,
  30,
  98,
  71,
  8,
  84,
  32,
  78,
  51,
  95,
  81,
  75,
  23,
  83,
  89,
  90,
  93,
  15,
  17,
  92,
  39,
  72,
  79,
  21,
  65,
  91,
  14],
 'bar': [57,
  22,
  66,
  1,
  35,
  74,
  6,
  25,
  60,
  11,
  62,
  65,
  49,
  44,
  54,
  59,
  12,
  42,
  36,
  52,
  31,
  77,
  41,
  64,
  9,
  76,
  55,
  16,
  2,
  17,
  13,
  68,
  34,
  24,
  4,
  56,
  10,
  63,
  0,
  51,
  38,
  19,
  70,
  43,
  26,
  47,
  32,
  72,
  3,
  61,
  21,
  29,
  75,
  50,
  15,
  5,
  39,
  14,
  37,
  7,
  67,
  46,
  33,
  20,
  45,
  53,
  18,
  

# Create full function

In [129]:
# Set `means_d`, `stdevs_d`
means_d = {}
stdevs_d = {}

for network in tqdm(representations_d, desc='mu, sigma'):
    t = representations_d[network]

    means_d[network] = t.mean(0, keepdim=True)
    stdevs_d[network] = (t - means_d[network]).pow(2).mean(0, keepdim=True).pow(0.5)

# Set `correlations`
correlations = {network: {} for network in representations_d} # {network: {other: tensor}}
num_words = list(representations_d.values())[0].size()[0] # TO DO: make more elegant

for network, other_network in tqdm(p(representations_d,
                                     representations_d),
                                   desc='correlate',
                                   total=len(representations_d)**2):

    if network == other_network:
        continue

    if other_network in correlations[network].keys(): # TO DO: optimize?
        continue

    t1 = representations_d[network] # "tensor"
    t2 = representations_d[other_network] 
    m1 = means_d[network] # "means"
    m2 = means_d[other_network]
    s1 = stdevs_d[network] # "stdevs"
    s2 = stdevs_d[other_network]

    covariance = (torch.mm(t1.t(), t2) / num_words # E[ab]
                  - torch.mm(m1.t(), m2)) # E[a]E[b]
    correlation = covariance / torch.mm(s1.t(), s2)
    
    correlation = correlation.cpu().numpy()
    correlations[network][other_network] = correlation
    correlations[other_network][network] = correlation.T

# Set `clusters`
clusters = {network: {} for network in representations_d} # {network: {neuron: {other: other_neuron}}}
for network in tqdm(representations_d, desc='clusters', total=len(representations_d)):
    for neuron in range(num_neurons_d[network]): 
        clusters[network][neuron] = {
            other : max(range(num_neurons_d[other]),
                         key = lambda i: abs(correlations[network][other][neuron][i])) 
             for other in correlations[network]
        }

neuron_sort = {} # {network, sorted_list}
# Sort neurons by worst (or best) best correlation with another neuron
# in another network.
for network in tqdm(representations_d, desc='annotation'):
    neuron_sort[network] = sorted(
        range(num_neurons_d[network]),
        key = lambda i : op(
            abs(correlations[network][other][i][clusters[network][i][other]])
            for other in clusters[network][i]),
        reverse=True
    )

correlate: 100%|█| 9/9 [00:00<00:00, 4087.13it/s]
